In [28]:
import boto3,time
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
## connnecting with dynamodb table and reading the content of table
TABLE_NAME = "dataset_info"
dynamodb = boto3.resource('dynamodb',
                          region_name="us-west-2"
)

table = dynamodb.Table(TABLE_NAME)

response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
## schema for the data extracted in sting format
cust_schema = """ dataset_key  string, created_ts  string, data_layer  string, data_location  string, dataset_id  string, dataset_refresh_end_ts  string, dataset_refresh_start_ts  string, full_load_start_ts  string, last_update_ts  string, run_mode  string, sla string, source_name  string, watermark_ts string """

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
## creating spark dataframe from dynamo table
df = spark.createDataFrame(data,cust_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
## doing processing in dataframe with some business logic...i.e adding columns using 'when' 'otherwise' statements

df = df.withColumn("data_location", when((df.data_location.isNull() | (df.data_location == ' ') | (df.data_location == '')) ,'null').otherwise(col("data_location")))
df = df.withColumn("full_load_start_ts", when((df.full_load_start_ts.isNull() | (df.full_load_start_ts == " ") | (df.full_load_start_ts == '')) ,"null").otherwise(col("full_load_start_ts")))
df = df.withColumn("run_mode", when((df.run_mode.isNull() | (df.run_mode == ' ') | (df.run_mode == '')) ,'null').otherwise(col("run_mode")))
df = df.withColumn("sla", when((df.sla.isNull() | (df.sla == ' ') | (df.sla == '')) ,'-1').otherwise(col("sla")))
df = df.withColumn("watermark_ts", when((df.watermark_ts.isNull() | (df.watermark_ts == ' ') | (df.watermark_ts == '')) ,'null').otherwise(col("watermark_ts")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
start = time.time()
df.coalesce(1).write.mode("overwrite").option("header","true").csv("s3://aws-poc-serverless-analytics/data/audit_tables/dataset_info/")
end = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
# to check the writing time in the s3 location
end-start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.6347332000732422

####  job_run_details

In [94]:
## reading table from dynamodb
TABLE_NAME = "job_run_details"

dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
table = dynamodb.Table(TABLE_NAME)
response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
cust_schema = """ error_message string, target_records string, dataset_id string, source string, job_key string, source_records string, dataset string, layer_name string, job_name string, end_ts string, start_ts string, job_run_date string, job_status string"""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
df = spark.createDataFrame(data,cust_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
df = df.withColumn("dataset_id",  when((df.dataset_id.isNull() | (df.dataset_id == "")| (df.dataset_id == " ")) ,'-1').otherwise(col("dataset_id")))
df = df.withColumn("target_records",  when((df.target_records.isNull() | (df.target_records == "")| (df.target_records == " ")) ,'-1').otherwise(col("target_records")))
df = df.withColumn("source_records",  when((df.source_records.isNull() | (df.source_records == "")| (df.source_records == " ")) ,'-1').otherwise(col("source_records")))
df = df.withColumn("start_ts", when((df.start_ts.isNull()| (df.start_ts == '')| (df.start_ts == ' ')),'-1').otherwise(col("start_ts")))
df = df.withColumn("end_ts", when((df.end_ts.isNull() | (df.end_ts == '')| (df.end_ts == ' ')) ,'-1').otherwise(col("end_ts")))
df = df.withColumn("source", when((df.source.isNull() | (df.source == '') | (df.source == ' ')),'null').otherwise(col("source")))
df = df.withColumn("dataset", when((df.dataset.isNull() | (df.dataset == '') | (df.dataset == ' ')),'null').otherwise(col("dataset")))
df = df.withColumn("layer_name", when((df.layer_name.isNull() | (df.layer_name == '') | (df.layer_name == ' ')),'null').otherwise(col("layer_name")))
df = df.withColumn("job_name", when((df.job_name.isNull() | (df.job_name == '') | (df.job_name == ' ')),'null').otherwise(col("job_name")))
df = df.withColumn("job_run_date", when((df.job_run_date.isNull() | (df.job_run_date == '') | (df.job_run_date == ' ')),'null').otherwise(col("job_run_date")))

df = df.withColumn("error_message", when((df.target_records.isNull() | (df.error_message == '') | (df.error_message == ' ')),'null').otherwise(col("error_message")))
df = df.withColumn("error_message", when(df.error_message != 'null',lit('Spark Error')).otherwise(col("error_message")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
start = time.time()
df.coalesce(1).write.mode("overwrite").option("header","true").csv("s3://aws-poc-serverless-analytics/data/audit_tables/job_run_details/")
end = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
end-start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13.380788564682007

#### integration of delta tables, pyspark with athena

In [11]:
%%configure -f
{ "conf":{    
          "spark.databricks.hive.metastore.glueCatalog.enabled" : "true",
          "spark.jars.packages": "io.delta:delta-core_2.12:0.8.0",
          "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
          "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
          "spark.sql.warehouse.dir": "s3://aws-poc-serverless-analytics/dev/clean/delta_dw/"
    
         }
}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1621492246389_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1621492246389_0008,pyspark,idle,Link,Link,✔


In [12]:
from delta.tables import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
deltaTable = DeltaTable.forPath(spark,'s3://aws-poc-serverless-analytics/dev/clean/clean_delta_lake_test/delta_lake_demo_new.db/merchants_clean')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df1 = spark.read.format("delta").load("s3://aws-poc-serverless-analytics/dev/clean/clean_delta_lake_test/delta_lake_demo_new.db/merchants_clean/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+----------+-----------+--------------------+--------------------+
|merchant_id|        company_name| telephone|      state|          created_at|          updated_at|
+-----------+--------------------+----------+-----------+--------------------+--------------------+
|      44082|Quick Draw Outdoo...|3182596422|       free|2019-02-04 16:57:...|2021-02-10 02:07:...|
|      44083|Quick Draw Outdoo...|3182596422|trial_ended|2019-02-04 16:57:...|2021-03-05 02:07:...|
|     156237|          Washer Fan|      null|provisioned|2020-10-26 20:10:...|2020-10-26 20:53:...|
|     176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|      50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|      44084|         godaddy llc|3182596422|provisioned|2019-02-04 16:57:...|2021-02-10 02:07:...|
|     144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|


In [30]:
deltaTable.generate("symlink_format_manifest")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
deltaTable.create

In [34]:
spark.sql("use delta_lake_Demo_new")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [45]:
spark.sql("CREATE EXTERNAL TABLE merchnats_externals (merchant_id string,company_name string,telephone long,created_at string,updated_at string)\
PARTITIONED BY (state string)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'\
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'\
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'\
LOCATION 's3://aws-poc-serverless-analytics/dev/clean/clean_delta_lake_test/delta_lake_demo_new.db/merchants_clean/_symlink_format_manifest/'")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]